In [1]:
import pandas as pd
import numpy as np

In [2]:
red_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', ';')

In [3]:
white_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', ';')

# Data preparation and analysis

In [4]:
# Having two different data set for Red and White wine produced by a winery company in the same region where 
# the consulting company wishes to establish, ergo the analysis.
# First step is to merge the data producing a new variable or category to determined the type of wine
# "red" or "white".

print(red_wine)

      fixed acidity  volatile acidity  citric acid  ...  sulphates  alcohol  quality
0               7.4             0.700         0.00  ...       0.56      9.4        5
1               7.8             0.880         0.00  ...       0.68      9.8        5
2               7.8             0.760         0.04  ...       0.65      9.8        5
3              11.2             0.280         0.56  ...       0.58      9.8        6
4               7.4             0.700         0.00  ...       0.56      9.4        5
...             ...               ...          ...  ...        ...      ...      ...
1594            6.2             0.600         0.08  ...       0.58     10.5        5
1595            5.9             0.550         0.10  ...       0.76     11.2        6
1596            6.3             0.510         0.13  ...       0.75     11.0        6
1597            5.9             0.645         0.12  ...       0.71     10.2        5
1598            6.0             0.310         0.47  ...       0.6

In [5]:
#create a new variable 'wine_type': "red" and "white" for an easier concatenation in ONE dataframe.

red_wine['wine_type'] = 'red'

In [6]:
white_wine['wine_type'] = 'white'

In [7]:
# In both  datasets there was already collected and provided a categorization on the wines produced. 
# from 1 - 9 where 9 is the highest and 1 the lowest quality.
# For an easier analysis and creation of a prediction model, the quality of wines would be re-organized
# in "low", "medium" and "high" quality labels.

red_wine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [8]:
# bucket "red" and "white" wine quality scores into qualitative quality labels

red_wine['quality_label'] = red_wine['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

In [9]:
red_wine['quality_label'] = pd.Categorical(red_wine['quality_label'],
categories=['low', 'medium', 'high'])

In [10]:
white_wine['quality_label'] = white_wine['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

In [11]:
white_wine['quality_label'] = pd.Categorical(white_wine['quality_label'],
categories=['low', 'medium', 'high'])

# we transform the new column from an object to a category in order to tell the machine not to order the data 
# in alphabetic order and let it know that all have "the same value"

In [12]:
#combine the two datasets into one : "wines"

wines = pd.concat([red_wine, white_wine])

In [13]:
# re-shuffle records just to randomize data points
wines = wines.sample(frac=1, random_state=42).reset_index(drop=True)

In [14]:
#Observe if there is missing values
wines.isnull().any()


fixed acidity           False
volatile acidity        False
citric acid             False
residual sugar          False
chlorides               False
free sulfur dioxide     False
total sulfur dioxide    False
density                 False
pH                      False
sulphates               False
alcohol                 False
quality                 False
wine_type               False
quality_label           False
dtype: bool

In [15]:
wines.isnull().sum().sum()

0

In [16]:
#save changes for each dataframe and the new combined one
red_wine.to_csv('red_wine.csv')


In [17]:
white_wine.to_csv("white_wine.csv")

In [ ]:
#save new dataframe
wines.to_csv('wines.csv')

In [ ]:
# One of the first observations from comparing "red" and "white" wine is:

In [18]:
# that  white wine is much more produced in the region and has very high quality wines in this category
table2 = pd.pivot_table(data=white_wine,index='quality_label',values='quality',aggfunc=np.sum)
print(table2)

               quality
quality_label         
low               7997
medium           19348
high              1445


In [19]:
#whilst red wine is less produced and mostly low to medium quality wine is expected from red type of wine
table3 = pd.pivot_table(data=red_wine,index='quality_label',values='quality',aggfunc=np.sum)
print(table3)

               quality
quality_label         
low               3647
medium            5221
high               144


In [20]:
# From the comparison from the ingrediants in "red" and "white" wine, one can easily observe that 
# "volatile acidity", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide"
# and "sulphates" present an important difference in levels in red and white type of wines.
wines.groupby("wine_type").mean('fixed acidity')

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
wine_type,,,,,,,,,,,,
red,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
white,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909


In [21]:
# taking "total sulfur dioxide" is a good way to example the above.
# and also a good way to observe that the less of this component in both red and white wine, the higher 
# the quality of the wine.
table = pd.pivot_table(data=red_wine,index='quality_label',values='total sulfur dioxide',aggfunc=np.sum)
print(table)

               total sulfur dioxide
quality_label                      
low                         40656.0
medium                      33044.0
high                          602.0


In [22]:
table1 = pd.pivot_table(data=white_wine,index='quality_label',values='total sulfur dioxide',aggfunc=np.sum)
print(table1)

               total sulfur dioxide
quality_label                      
low                        243700.5
medium                     411331.0
high                        22659.0


In [ ]:
#Given the relationship between the components of the wine and its quality. The next step is to visualize
# the mentioned observation. 